In [1]:
import os
import pandas as pd

base_path = r"C:\Users\anoth\Downloads\404GX_Social-Security-Data-main\Trustee Report (Cleaned)"
base_path


'C:\\Users\\anoth\\Downloads\\404GX_Social-Security-Data-main\\Trustee Report (Cleaned)'

In [2]:
import pandas as pd
import glob
from functools import reduce

files = glob.glob(base_path + "/*.csv")

dfs = []

for f in files:
    try:
        temp = pd.read_csv(f, encoding="utf-8")
    except UnicodeDecodeError:
        temp = pd.read_csv(f, encoding="latin1")
    
    if "Year" in temp.columns:
        temp["Year"] = temp["Year"].astype(str).str.strip()
        dfs.append(temp)

len(dfs)


10

In [3]:
# start with the first dataframe
merged = dfs[0].copy()

for temp in dfs[1:]:
    # find overlapping columns OTHER than Year
    overlap = [c for c in temp.columns if c in merged.columns and c != "Year"]
    
    # drop those overlapping columns from the new df before merge
    if overlap:
        temp = temp.drop(columns=overlap)
    
    # now safe to merge
    merged = merged.merge(temp, on="Year", how="outer")

df = merged
df.columns


Index(['Assumption', 'Year', 'Average Annual Unemployment Rate',
       'Labor Force Percentage Change', 'Total Employment Percentage Change',
       'GDP Percentage Change', 'Average Annual Nominal Interest Rate',
       'Average Annual Real Interest Rate', 'Covered workers',
       'OASI Beneficiaries', 'DI Beneficiaries', 'OASDI Beneficiaries',
       'Covered workers per OASDI beneficiary',
       'OASDI beneficiaries per 100 covered workers', 'Total Fertility Rate',
       'Age-sex-adjusted death rate per 100,000',
       'Age-sex-adjusted death rate per 100,000 (Under 65)',
       'Age-sex-adjusted death rate per 100,000 (65 and over)',
       'LPR Immigration', 'LPR Emigration', 'Net LPR Change',
       'Net Temporary or Unlawful Immigration Change', 'Income rate (OASDI)',
       'Cost rate (OASDI)', 'Balance (OASDI)', 'Income rate (HI)',
       'Cost rate (HI)', 'Balance (HI)',
       'Worker (Retired workers & auxiliaries)',
       'Spouse (Retired workers & auxiliaries)',
   

In [4]:
# convert year + filter
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df = df[df["Year"] <= 2024].copy()

# collapse to unique years
df = df.groupby("Year", as_index=False).mean(numeric_only=True)


# create covid dummy
df["Covid_dummy"] = (df["Year"] == 2020).astype(int)



In [5]:
df["LE65_x_Covid"] = df["Life Expectancy at Age 65"] * df["Covid_dummy"]

In [6]:
import statsmodels.api as sm

In [7]:
# outcome (dependent) variables
outcome_cols = [
    "Reserves at end of year (K)",
    "Cost (K)",
    "OASDI Fund Ratio"
]

# columns to exclude from predictors
exclude_cols = set(outcome_cols + ["Year", "Assumption"])

# everything else will be a potential X
predictor_cols = [c for c in df.columns if c not in exclude_cols]

len(predictor_cols), predictor_cols[:10]


(43,
 ['Average Annual Unemployment Rate',
  'Labor Force Percentage Change',
  'Total Employment Percentage Change',
  'GDP Percentage Change',
  'Average Annual Nominal Interest Rate',
  'Average Annual Real Interest Rate',
  'Covered workers',
  'OASI Beneficiaries',
  'OASDI Beneficiaries',
  'Covered workers per OASDI beneficiary'])

In [8]:
results = []

for y_col in outcome_cols:
    for x_col in predictor_cols:
        # take just these two columns
        sub = df[[y_col, x_col]].copy()

        # force both columns to numeric, invalid values → NaN
        sub = sub.apply(pd.to_numeric, errors="coerce").dropna()

        # skip if too few data points or not enough variation in X
        if len(sub) < 10 or sub[x_col].nunique() < 3:
            continue

        X = sm.add_constant(sub[x_col])
        y = sub[y_col]

        model = sm.OLS(y, X).fit()

        results.append({
            "outcome": y_col,
            "predictor": x_col,
            "beta": model.params[x_col],
            "p_value": model.pvalues[x_col],
            "r_squared": model.rsquared,
            "n_obs": len(sub)
        })

results_df = pd.DataFrame(results)
results_df.head()


,outcome,predictor,beta,p_value,r_squared,n_obs
0,Reserves at end of year (K),Average Annual Unemployment Rate,-1.921634e+08,1.303456e-01,0.042649,55
1,Reserves at end of year (K),Labor Force Percentage Change,-1.115981e+09,8.016235e-09,0.469211,55
2,Reserves at end of year (K),Total Employment Percentage Change,-2.658844e+08,2.284262e-02,0.093952,55
3,Reserves at end of year (K),GDP Percentage Change,-1.798782e+08,7.566559e-02,0.058331,55
4,Reserves at end of year (K),Average Annual Nominal Interest Rate,-4.378756e+08,7.698956e-20,0.794300,55


In [9]:
results_df_sorted = results_df.sort_values(["outcome", "p_value"])
results_df_sorted


,outcome,predictor,beta,p_value,r_squared,n_obs
49,Cost (K),OASDI Beneficiaries,2.862999e+04,1.917422e-62,0.994903,55
48,Cost (K),OASI Beneficiaries,3.573644e+04,7.070810e-51,0.986077,55
66,Cost (K),Worker (Retired workers & auxiliaries),3.320222e+04,5.714296e-50,0.990288,50
73,Cost (K),Total,3.638035e+04,1.491671e-45,0.985164,50
74,Cost (K),Non-interest income (K),1.054629e+00,3.054957e-33,0.935612,55
...,...,...,...,...,...,...
3,Reserves at end of year (K),GDP Percentage Change,-1.798782e+08,7.566559e-02,0.058331,55
21,Reserves at end of year (K),Balance (OASDI),-2.778981e+08,1.200241e-01,0.044991,55
0,Reserves at end of year (K),Average Annual Unemployment Rate,-1.921634e+08,1.303456e-01,0.042649,55
18,Reserves at end of year (K),Net Temporary or Unlawful Immigration Change,5.793770e+05,1.707399e-01,0.035108,55


In [10]:
focus_names = [
    "Total Fertility Rate",
    "Life Expectancy at Birth",
    "Life Expectancy at Birth, Male",
    "Life Expectancy at Birth, Female",
    "Net LPR Change"
]

focus_cols = [c for c in focus_names if c in predictor_cols]

focus_cols


['Total Fertility Rate', 'Life Expectancy at Birth', 'Net LPR Change']

In [11]:
focus_results = results_df_sorted[results_df_sorted["predictor"].isin(focus_cols)]
focus_results


,outcome,predictor,beta,p_value,r_squared,n_obs
77,Cost (K),Life Expectancy at Birth,1.324313e+08,6.396261e-20,0.795728,55
58,Cost (K),Net LPR Change,6.739309e+05,2.659636e-05,0.285395,55
52,Cost (K),Total Fertility Rate,-8.793399e+08,1.285260e-03,0.179106,55
118,OASDI Fund Ratio,Life Expectancy at Birth,6.052626e-01,8.478410e-19,0.807527,50
99,OASDI Fund Ratio,Net LPR Change,2.089047e-03,1.911098e-03,0.183520,50
93,OASDI Fund Ratio,Total Fertility Rate,1.583087e+00,2.056005e-01,0.033164,50
36,Reserves at end of year (K),Life Expectancy at Birth,5.891452e+08,7.058229e-19,0.776454,55
17,Reserves at end of year (K),Net LPR Change,2.855016e+06,9.243779e-05,0.252534,55
11,Reserves at end of year (K),Total Fertility Rate,-1.135344e+09,3.775554e-01,0.014721,55


In [12]:
# Special COVID model:
sub = df[["OASDI Fund Ratio", "Life Expectancy at Age 65", "Covid_dummy"]].copy()
sub = sub.apply(pd.to_numeric, errors="coerce").dropna()

X = sm.add_constant(sub[["Life Expectancy at Age 65", "Covid_dummy"]])
y = sub["OASDI Fund Ratio"]

covid_model = sm.OLS(y, X).fit()
print(covid_model.summary())


                            OLS Regression Results                            
Dep. Variable:       OASDI Fund Ratio   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     71.93
Date:                Sun, 07 Dec 2025   Prob (F-statistic):           4.98e-15
Time:                        18:46:18   Log-Likelihood:                -46.020
No. Observations:                  50   AIC:                             98.04
Df Residuals:                      47   BIC:                             103.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [13]:
# ---- Interaction regression only for OASDI Fund Ratio ----
sub = df[[
    "OASDI Fund Ratio",
    "Life Expectancy at Age 65",
    "Covid_dummy",
    "LE65_x_Covid"
]].copy()

sub = sub.apply(pd.to_numeric, errors="coerce").dropna()

X = sm.add_constant(sub[[
    "Life Expectancy at Age 65",
    "Covid_dummy",
    "LE65_x_Covid"
]])
y = sub["OASDI Fund Ratio"]

interaction_model = sm.OLS(y, X).fit()
print(interaction_model.summary())


                            OLS Regression Results                            
Dep. Variable:       OASDI Fund Ratio   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     71.93
Date:                Sun, 07 Dec 2025   Prob (F-statistic):           4.98e-15
Time:                        18:46:18   Log-Likelihood:                -46.020
No. Observations:                  50   AIC:                             98.04
Df Residuals:                      47   BIC:                             103.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [14]:
results_df_sorted.to_csv("all_univariate_results.csv", index=False)
focus_results.to_csv("focus_univariate_results.csv", index=False)


In [15]:
df.head

<bound method NDFrame.head of       Year  Average Annual Unemployment Rate  Labor Force Percentage Change  \
0   1940.0                               NaN                            NaN   
1   1941.0                               NaN                            NaN   
2   1942.0                               NaN                            NaN   
3   1943.0                               NaN                            NaN   
4   1944.0                               NaN                            NaN   
..     ...                               ...                            ...   
80  2020.0                               8.1                           -1.7   
81  2021.0                               5.4                            0.3   
82  2022.0                               3.6                            1.9   
83  2023.0                               3.6                            1.7   
84  2024.0                               4.1                            0.7   

    Total Employment 

In [16]:
df.to_csv("final_clean_dataset.csv", index=False)